<a href="https://colab.research.google.com/github/Guy-Ben-Yosef/094219_Final_Exam_Preparation/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install faster_whisper

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 37.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.7/39.7 MB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 38.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 3.2 MB/s eta 0:00:00


In [2]:
!wget -O audio.mp3 !wget -O audio.mp3 https://raw.githubusercontent.com/Guy-Ben-Yosef/makore/main/audio/Shauli.mp3

--2025-03-22 13:37:12--  http://!wget/
Resolving !wget (!wget)... failed: Name or service not known.
wget: unable to resolve host address ‘!wget’
--2025-03-22 13:37:12--  https://raw.githubusercontent.com/Guy-Ben-Yosef/makore/main/audio/Shauli.mp3
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 635461 (621K) [audio/mpeg]
Saving to: ‘audio.mp3’

audio.mp3           100%[===================>] 620.57K  --.-KB/s    in 0.004s  

2025-03-22 13:37:13 (138 MB/s) - ‘audio.mp3’ saved [635461/635461]

FINISHED --2025-03-22 13:37:13--
Total wall clock time: 0.8s
Downloaded: 1 files, 621K in 0.004s (138 MB/s)


In [3]:
from IPython.display import Audio, display

display(Audio('audio.mp3', autoplay=True))

In [ ]:
"""import faster_whisper

model = faster_whisper.WhisperModel('ivrit-ai/faster-whisper-v2-d4')

segs, _ = model.transcribe('audio.mp3', language='he')

texts = [s.text for s in segs]

transcribed_text = ' '.join(texts)
print(f'Transcribed text: {transcribed_text}')"""

Transcribed text:  אני שאולי, יזם, איש יזמות, התעסקתי שנים ביבוא נדלן, מה הסוד שלי אתה שואל? אני... לא שאלתי, לא משנה, לא שאלתי. אתה תשאל תכף, תראה. אז אני אענה לך. אוקיי. הסוד שלי הוא שאני עובד מהלב.


---
---

In [4]:
import faster_whisper
import time
from datetime import datetime
import json
from pathlib import Path
import pandas as pd
from tqdm.notebook import tqdm
import os

In [5]:
# config
INPUT_FILE = 'audio.mp3'
LANGUAGE = 'he'
MODEL_NAME = 'ivrit-ai/faster-whisper-v2-d4'

# output dirs
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
output_dir = Path(f'transcription_results_{timestamp}')
output_dir.mkdir(exist_ok=True)

# model init
print("Loading model...")
start_time = time.time()
model = faster_whisper.WhisperModel(MODEL_NAME)
model_load_time = time.time() - start_time
print(f"Model loaded in {model_load_time:.2f} seconds")

Loading model...


tokenizer.json:   0%|          | 0.00/2.20M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/339 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/12.1k [00:00<?, ?B/s]

vocabulary.json:   0%|          | 0.00/1.07M [00:00<?, ?B/s]

model.bin:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

Model loaded in 138.49 seconds


In [8]:
def save_segment(segment, file_handle, json_file):
    file_handle.write(f"{segment.start:.2f} -- {segment.end:.2f} -- {segment.text}\n")
    file_handle.flush()

    # format as dictionary
    segment_dict = {
        'start': segment.start,
        'end': segment.end,
        'text': segment.text,
        'timestamp': datetime.now().isoformat()
    }
    json_file.write(json.dumps(segment_dict, ensure_ascii=False) + '\n')
    json_file.flush()

    return segment_dict

def print_transcription_results(output_dir):
    """
    Prints the transcription results in a nicely formatted way.

    Args:
        output_dir (Path): Path to the directory containing the transcription results
    """
    # Load the transcription data
    json_path = output_dir / 'transcription.jsonl'
    metrics_path = output_dir / 'metrics.json'

    # Check if files exist
    if not json_path.exists() or not metrics_path.exists():
        print(f"Error: Required files not found in {output_dir}")
        return

    # Load metrics
    with open(metrics_path, 'r', encoding='utf-8') as f:
        metrics = json.load(f)

    # Load segments
    segments = []
    with open(json_path, 'r', encoding='utf-8') as f:
        for line in f:
            segments.append(json.loads(line))

    # Print header
    print("\n" + "=" * 80)
    print(f"TRANSCRIPTION RESULTS")
    print("=" * 80)

    # Print metadata
    print(f"\nFile: {metrics['input_file']}")
    print(f"Model: {metrics['model_name']}")
    print(f"Language: {metrics['language']}")
    print(f"Processing time: {metrics['transcription_time_seconds']:.2f} seconds")
    print(f"Total segments: {metrics['total_segments']}")
    print(f"Completed at: {metrics['timestamp']}")

    # Print full transcript
    print("\n" + "-" * 80)
    print("FULL TRANSCRIPT")
    print("-" * 80)

    full_text = ' '.join([segment['text'] for segment in segments])
    print(f"\n{full_text}\n")

    # Print segments with timestamps
    print("-" * 80)
    print("SEGMENTS WITH TIMESTAMPS")
    print("-" * 80 + "\n")

    for i, segment in enumerate(segments, 1):
        start = segment['start']
        end = segment['end']
        duration = end - start

        # Format timestamps as MM:SS.ms
        start_fmt = f"{int(start // 60):02d}:{start % 60:06.3f}"
        end_fmt = f"{int(end // 60):02d}:{end % 60:06.3f}"

        print(f"[{i:03d}] {start_fmt} → {end_fmt} ({duration:.2f}s)")
        print(f"     {segment['text']}")
        print()

    print("=" * 80)
    print(f"End of transcription for {metrics['input_file']}")
    print("=" * 80)

In [7]:
print(f"Starting transcription of {INPUT_FILE}...")
transcription_start = time.time()

txt_path = output_dir / 'transcription.txt'
json_path = output_dir / 'transcription.jsonl'
metrics_path = output_dir / 'metrics.json'

segments_data = []

with open(txt_path, 'w', encoding='utf-8') as txt_file, \
     open(json_path, 'w', encoding='utf-8') as json_file:

    # headers
    txt_file.write(f"Transcription of {INPUT_FILE}\n")
    txt_file.write(f"Started at: {datetime.now()}\n")
    txt_file.write("-" * 50 + "\n")

    ## TRANSCRIPTION ##
    segments, info = model.transcribe(INPUT_FILE, language=LANGUAGE)

    # process with pb
    for segment in tqdm(segments, desc="Processing segments"):
        segment_data = save_segment(segment, txt_file, json_file)
        segments_data.append(segment_data)

# timedelta
transcription_time = time.time() - transcription_start

# save into dictionary
metrics = {
    'input_file': INPUT_FILE,
    'model_name': MODEL_NAME,
    'model_load_time_seconds': model_load_time,
    'transcription_time_seconds': transcription_time,
    'total_segments': len(segments_data),
    'language': LANGUAGE,
    'timestamp': datetime.now().isoformat(),
    # 'info': info._asdict() if info else {}
}

with open(metrics_path, 'w', encoding='utf-8') as f:
    json.dump(metrics, f, ensure_ascii=False, indent=2)

Starting transcription of audio.mp3...


Processing segments: 0it [00:00, ?it/s]

In [12]:
print_transcription_results(output_dir)


TRANSCRIPTION RESULTS

File: audio.mp3
Model: ivrit-ai/faster-whisper-v2-d4
Language: he
Processing time: 4.31 seconds
Total segments: 1
Completed at: 2025-03-22T13:40:54.896894

--------------------------------------------------------------------------------
FULL TRANSCRIPT
--------------------------------------------------------------------------------

 אני שאולי, יזם, איש יזמות, התעסקתי שנים ביבוא נדלן, מה הסוד שלי אתה שואל? אני... לא שאלתי, לא משנה, לא שאלתי. אתה תשאל תכף, תראה. אז אני אענה לך. אוקיי. הסוד שלי הוא שאני עובד מהלב.

--------------------------------------------------------------------------------
SEGMENTS WITH TIMESTAMPS
--------------------------------------------------------------------------------

[001] 00:00.000 → 00:15.060 (15.06s)
      אני שאולי, יזם, איש יזמות, התעסקתי שנים ביבוא נדלן, מה הסוד שלי אתה שואל? אני... לא שאלתי, לא משנה, לא שאלתי. אתה תשאל תכף, תראה. אז אני אענה לך. אוקיי. הסוד שלי הוא שאני עובד מהלב.

End of transcription for audio.mp3
